# An attempt at using the cogent API
An attempt made by an electromechanical engineer, not a programmer, so be warned

## First aid with Linked Data Event Stream
The collection is semantically stored: there is no fixed data model necessary. It is possible to access the data from different angels.

Depending on the time you have & your interest, I curated some articles on this approach:

* If you know nothing about LDES or semantic descriptions, following article explains it in 8 minutes https://medium.com/@samuel.vanackere/linked-data-event-streams-explained-in-8-minutes-e1c76d077bb9 
* If you want to know how to backend of the collection of Ghent functions, you can view following 6 minutes video: https://youtu.be/8_0QSEj_Qhg
* If you want to have the broader picture (why does LDES exists, where does it fit in within the internet landscape, who is building it...), check out following course (+-2 hours): https://academy.europa.eu/course/view.php?id=239 





## LDES event stream of the collection of Ghent

The base URL of the collection is https://apidg.gent.be/opendata/adlib2eventstream/v1/

I'll try Python to demonstrate some functionalities. Best is to run this notebook within Visual Studio Code: it allows you to open the prints in another window.

In [1]:
import httpx 
import json

# There exists multiple implementations of jsonpath for Python, not all of them have the ability to filter. 
# Some implementations throw an error if you try to use a filter funcitonallity within JSON path

from jsonpath_ng.ext import parser 

In [2]:
# The base url points to a page that has an overview of all the datasets
base = "https://apidg.gent.be/opendata/adlib2eventstream/v1/"
datasets = httpx.get(base).json()

print(json.dumps(datasets,indent=4))


{
    "@context": [
        "https://apidg.gent.be/opendata/adlib2eventstream/v1/context/DCAT-AP-VL.jsonld",
        {
            "dcterms": "http://purl.org/dc/terms/",
            "ldes": "https://w3id.org/ldes#",
            "tree": "https://w3id.org/tree#",
            "tree:view": {
                "@type": "@id"
            }
        }
    ],
    "@id": "https://stad.gent/id/dcat/coghent",
    "@type": "Datasetcatalogus",
    "Datasetcatalogus.titel": {
        "@value": "Catalogus CoGhent",
        "@language": "nl"
    },
    "Datasetcatalogus.beschrijving": {
        "@value": "Catalogus van datasets voor de Collectie van de Gentenaar.",
        "@language": "nl"
    },
    "Datasetcatalogus.heeftLicentie": {
        "@id": "https://creativecommons.org/publicdomain/zero/1.0/"
    },
    "Datasetcatalogus.heeftUitgever": {
        "@id": "http://stad.gent/",
        "Agent.naam": {
            "@value": "Stad Gent",
            "@language": "nl"
        }
    },
    "Datasetca

In [7]:
print("All datasets:")
dataset_names = parser.parse('$..["Dataset.titel"]').find(datasets)
for match in dataset_names:
    print(match.value["@value"])

All datasets:
thesaurus van Archief Gent, Design Museum Gent, Het Huis van Alijn (Gent), Industriemuseum, STAM (Gent)
personen van Archief Gent, Design Museum Gent, Het Huis van Alijn (Gent), Industriemuseum, STAM (Gent)
objecten van Archief Gent
tentoonstellingen van Design Museum Gent
objecten van Design Museum Gent
objecten van Het Huis van Alijn (Gent)
objecten van Industriemuseum
objecten van STAM (Gent)


In [8]:
# Let's pick out a dataset of the catalog:
# TODO: make a function to get all datasets
dataset = datasets["Datasetcatalogus.heeftDataset"][6]

print(dataset["Dataset.beschrijving"])
print(dataset["tree:view"])

{'@value': "Event stream van de Adlib database 'objecten' van de instelling: Industriemuseum", '@language': 'nl'}
https://apidg.gent.be/opendata/adlib2eventstream/v1/industriemuseum/objecten


In [9]:
# get the first part of the dataset, the dataset works with a continuation link, 
# so if a request is to big to do it within one http request, the server will spit out
# TODO: write code to automatically stitch together continuation links
objects = httpx.get(dataset["tree:view"],follow_redirects=True).json()
print(json.dumps(objects,indent=4))

{
    "@context": {
        "prov": "http://www.w3.org/ns/prov#",
        "tree": "https://w3id.org/tree#",
        "sh": "http://www.w3.org/ns/shacl#",
        "dcterms": "http://purl.org/dc/terms/",
        "tree:member": {
            "@type": "@id"
        },
        "memberOf": {
            "@reverse": "tree:member",
            "@type": "@id"
        },
        "tree:node": {
            "@type": "@id"
        },
        "tree:path": {
            "@type": "@id"
        },
        "viewOf": {
            "@reverse": "tree:view",
            "@type": "@id"
        }
    },
    "@id": "https://apidg.gent.be/opendata/adlib2eventstream/v1/industriemuseum/objecten?generatedAtTime=2023-03-23T00:08:31.465Z",
    "@type": "tree:Node",
    "viewOf": {
        "@id": "https://stad.gent/id/dataset/industriemuseum/423276211afbdf9f78f02d580c2dcd58",
        "@type": "tree:Collection"
    },
    "@included": [
        {
            "@context": [
                "https://apidg.gent.be/opendata

In [11]:
# Get all the objects that are a digital object. 
# A digital object contains an ID that points to a webpage that contains all the information of that digital object (description, jpeg)
matches = parser.parse('$..["Entiteit.isHetOnderwerpVan"][?(@.["@type"]=="DigitalObject")]').find(objects)
print(f"Number of objects found: {len(matches)}")
for match in matches:
    print(match.value["@id"])

Number of objects found: 93
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V27713-001
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V27715
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V25084
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V27714
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V29943
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V33209
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V08899
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V08897
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V08898
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V08900
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V09336
https://api.collectie.gent/iiif/presentation/v2/manifest/industriemuseum:V10147
https://